In [4]:
#import 

import os
from dotenv import load_dotenv
import voyageai
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import openai
import numpy as np


In [5]:
#load .env file and apikey

load_dotenv()

api_key = os.getenv("VOYAGI_AI_API_KEY")
api_key_OPENAI = os.getenv("OPENAI_API_KEY")

In [6]:
#test documents for the vectordatabase

documents = ['Renas  is from kurdistan and has a cat', 
             'Renas  is a kurdish guerilla fighter, he is fighting for kurdish independence', 
             'Hassan Saado i a swedish gangster who can bench 150kg',
              "Renas favorite food in runescape is burnt lobster and he likes to eat it every fridag.", 
              "Renas and Hassan meet when studying at Uppsala university, they studied chemistry in starting 2016, but they both dropped out to pursue computer science." ]



In [7]:
# Initialize the client and embedd the documents
vo = voyageai.Client(api_key)

batch_size = 128
documents_emb = [
    vo.embed(
        documents[i:i + batch_size], model="voyage-2", input_type="document",
    ).embeddings
    for i in range(0, len(documents), batch_size)
]

In [8]:
query = "when did hassan an renas start studying?"

# Get the embedding of the query
query_emb = vo.embed(
    [query], model="voyage-2", input_type="query"
).embeddings[0]

In [9]:
distances_list = []

for i in range(len(documents_emb[0])):

        list_first_doc = np.array(documents_emb[0][i])
        query_emb1 = np.array(query_emb)
        result = cosine_similarity(list_first_doc.reshape(1, -1),query_emb1.reshape(1, -1))
        print(type(result))
        result = float(result)
        distances_list.append(result)

index_of_max = distances_list.index(max(distances_list))


# Initialize OpenAI API

load_dotenv()


# Take the retrieved document and use it as a prompt for the text generation model
prompt = f"Based on the information: '{documents[index_of_max]}', {query}"

from openai import OpenAI
client = OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)
   

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
ChatCompletionMessage(content='Renas and Hassan started studying at Uppsala University in early 2016.', role='assistant', function_call=None, tool_calls=None)
